## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [2]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [5]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder(categories='auto')
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [6]:

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32, (3, 3), input_shape=(32, 32 ,3), activation='relu'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
#classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(64, (3, 3)))
classifier.add(BatchNormalization())

classifier.add(MaxPooling2D(pool_size=(2,2)))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(units=100, activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)

C:\Users\alway\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`
Epoch 1/100
50000/50000 [==============================] - 145s 3ms/step - loss: 1.4985 - accuracy: 0.4720
Epoch 2/100
50000/50000 [==============================] - 154s 3ms/step - loss: 1.0655 - accuracy: 0.6224
Epoch 3/100
50000/50000 [==============================] - 151s 3ms/step - loss: 0.9168 - accuracy: 0.6759
Epoch 4/100
50000/50000 [==============================] - 150s 3ms/step - loss: 0.8193 - accuracy: 0.7133
Epoch 5/100
50000/50000 [==============================] - 157s 3ms/step - loss: 0.7365 - accuracy: 0.7450
Epoch 6/100
50000/50000 [==============================] - 165s 3ms/step - loss: 0.6513 - accuracy: 0.7730
Epoch 7/100
50000/50000 [==============================] - 199s 4ms/step - loss: 0.5763 - accuracy: 0.7976
Epoch 8/100
50000/50000 [==============================

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [7]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[1.27879776e-25, 0.00000000e+00, 1.45953929e-21, 9.99999642e-01,
        3.45492055e-07, 0.00000000e+00, 1.09478085e-35, 6.52303583e-36,
        2.45240001e-31, 0.00000000e+00]], dtype=float32)